In [2]:
import numpy as np
import pandas as pd


from src.datamodules import AASISTDataset

In [3]:
train_df = pd.read_csv("../dataset/train.csv")
test_df = pd.read_csv("../dataset/test.csv")

In [4]:
train_dset = AASISTDataset(df=train_df, train_mode=True)
test_dset  = AASISTDataset(df=test_df, train_mode=False)

In [6]:
train_dset[10]

(tensor([-0.0377, -0.0200,  0.0209,  ..., -0.2023, -0.1641, -0.1303]),
 (1.0, 0.0))